# Libraries


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import datetime
import openai


# 1. Introduction

In this project, our aim is to implement a number of machine learning models to answer  our research question on the AI Tools  dataset.

- Research Question: Which predictors and criterion contribute to a good predictive model for AI tool success?

During this theis project we consolidate the knowledge by building a variety of machine learning models independently.

# Testing categorization with multiple natural language options

### Based on keywords

In [94]:
category_keywords = {
    'Digital Model of Physical Objects': ['3D model', 'CAD', 'digital twin', 'simulation'],
    'Data': ['data', 'database', 'big data', 'analytics', 'data mining'],
    'Process/Workflow': ['workflow', 'process', 'automation', 'BPM', 'business process'],
    'Text': ['text', 'NLP', 'natural language processing', 'document', 'content'],
    'Multimodal': ['multimodal', 'mixed media', 'multiple inputs', 'sensor fusion'],
    'Code': ['code', 'programming', 'source code', 'repository', 'version control'],
    'Data and Analytics': ['business intelligence', 'reporting', 'data analysis', 'visualization'],
    'Sound': ['sound', 'audio', 'music', 'acoustic', 'sonic'],
    'Video': ['video', 'film', 'footage', 'streaming', 'broadcast'],
    'Image': ['image', 'picture', 'photo', 'graphics', 'visualization'],
    'NA/Ops/Support': ['operations', 'support', 'maintenance', 'logistics', 'infrastructure']
}

def label_description(description, category_keywords):
    # Standaard label als geen sleutelwoorden gevonden worden
    label = 'Uncategorized'
    
    # Zoek door elke categorie en bijbehorende sleutelwoorden
    for category, keywords in category_keywords.items():
        if any(keyword.lower() in description.lower() for keyword in keywords):
            label = category
            break  # Stop als een sleutelwoord gevonden is
    
    return label

# Pas de labelbeschrijvingsfunctie toe op elke rij in de dataframe
predicted_labels = df['cleaned_description'].apply(lambda desc: label_description(desc, category_keywords))

# Voeg de voorspelde labels toe aan de dataframe
df['predicted_label'] = predicted_labels

# Bereken de nauwkeurigheid als het percentage van de juiste voorspellingen
accuracy = (df['predicted_label'] == df['type of output modality']).mean()

print(f'Accuracy of keyword-based classification: {accuracy}')
accuracy

Accuracy of keyword-based classification: 0.2454308093994778


0.2454308093994778

In [96]:
import numpy as np

def check_keywords(description, category_keywords):
    # Controleer of er ten minste één sleutelwoord in de beschrijving staat
    for keywords in category_keywords.values():
        if any(keyword.lower() in description.lower() for keyword in keywords):
            return description  # Keer terug met de originele beschrijving als een sleutelwoord gevonden is
    return np.nan  # Keer terug met NaN als geen sleutelwoorden gevonden zijn

# Pas de functie toe op elke beschrijving in de dataframe
df['description_with_keywords'] = df['cleaned_description'].apply(lambda desc: check_keywords(desc, category_keywords))

# Controleer hoeveel beschrijvingen NaN zijn omdat ze geen sleutelwoorden bevatten
nan_count = df['description_with_keywords'].isna().sum()
print(f'Aantal beschrijvingen zonder sleutelwoorden: {nan_count}')


Aantal beschrijvingen zonder sleutelwoorden: 93


In [97]:
#how many descriptions 
print(df['description_with_keywords'].notnull().sum())

290


The accuracy was low so the decision was made to select based on the API of OpenAI to use for classifying the tools for its use case.

### Classifying with Natural Language Processing OpenAI API

####  OpenAI classifier costs money, so not decided to do

In [ ]:
def classify_descriptions(data):
    descriptions = data['desc'].tolist()

    category_predictions = []
    for description in descriptions:  
        try:
            completion = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are an assistant. Your task is to classify product descriptions into categories. Respond with only the category number."},
                    {"role": "user", "content": description}
                ]
            )
            category_number = completion.choices[0].message['content'].strip()
            category_predictions.append(category_number)
        except Exception as e:
            print(f"Er is een fout opgetreden: {e}")
            category_predictions.append("Error")

    return category_predictions

category_preds = classify_descriptions(data)
data['predicted_category'] = category_preds



Adding amount of review per tool

In [ ]:
# Tel het aantal reviews per tool_id
review_counts = data2.groupby('tool_id')['review_id'].count()

# Pak de eerste all_time_score per tool_id
first_scores = data2.groupby('tool_id')['all_time_score'].first()

# Voeg deze informatie toe aan new_df
new_df['review_count'] = new_df['tool_id'].map(review_counts)
new_df['first_all_time_score'] = new_df['tool_id'].map(first_scores)

# Conclusion


We will continue with another way of processing large language models. OpenAI costs money and keyword filtering is not working properly. 